**Задание 0. Загрузите необходимые библиотеки**

In [32]:
!pip install nltk spacy
!pip install pymystem3
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_sm
import spacy
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 88.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Задание 1. Загрузка данных**

Вставьте текст для обработки согласно вашему варианту

In [59]:
text = """В 2025 году команда ResearchLab_42 запустила пилотный проект Digital Culture Mapping,
цель которого — анализ user-generated content (UGC) из соцсетей и мессенджеров (Telegram, X/Twitter, VK).
По словам project manager’а, KPI выросли на 37% по сравнению с Q4 2024, несмотря на budget cut в −15%.
Основные данные собирались через API, затем обрабатывались в Python (NLTK, spaCy, pymystem3).
В логах часто появлялись строки вида: [WARN][12:45:07] Token "update2025_version2" skipped. Внутренние отчёты
сохранялись как PDF (size ≈ 2.4 MB) и DOCX-файлы. Интересно, что в текстах часто встречались гибридные конструкции:
fake news, инфодемия, post-truth эпоха, fact-checking, soft power, hard skills vs. soft skills.
Некоторые респонденты писали: “Это было totally unexpected”, “Я апдейтнул файл вчера в 23:59”,
“Meeting перенесли на Monday”. В итоговом summary (≈ 1 250 words) отмечалось, что частотность аббревиатур
типа AI, NLP, LLM, GPT-4, EU, USA превышает норму для академического дискурса, что подтверждает гипотезу о гибридизации научного и медийного регистров."""

**Задание 2. Нормализация текста.**

Приведите текст к нижнему регистру, удалите лишние пробоелы, переносы строк, спецсимволы, пунктуацию, обработайте цифры.

In [48]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[©®™℠]+', ' ', text)
    text = re.sub(r'\([^)]*\)', ' ', text)
    text = text.replace('ё', 'е')
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)
    text = re.sub(r'HYPHEN', '-', text)
    text = re.sub(r'\b\d+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

normalized_text = normalize_text(text)
print(normalized_text)

я апдейтнул файл вчера в meeting перенесли на monday в итоговом summary отмечалось что частотность аббревиатур типа ai nlp llm gpt- eu usa превышает норму для академического дискурса что подтверждает гипотезу о гибридизации научного и медийного регистров


**Задание 3. Токенизация**

Токенизируйте текст.

In [35]:
def tokenize(normalized_text):
    tokens = re.findall(r'\b[\w\-]+\b', normalized_text)

    return tokens

tokenized_text = tokenize(normalized_text)
print(tokenized_text)

['в', 'году', 'команда', 'researchlab_42', 'запустила', 'пилотный', 'проект', 'digital', 'culture', 'mapping', 'цель', 'которого', 'анализ', 'user-generated', 'content', 'из', 'соцсетей', 'и', 'мессенджеров', 'по', 'словам', 'project', 'manager', 'а', 'kpi', 'выросли', 'на', 'по', 'сравнению', 'с', 'q4', 'несмотря', 'на', 'budget', 'cut', 'в', 'основные', 'данные', 'собирались', 'через', 'api', 'затем', 'обрабатывались', 'в', 'python', 'в', 'логах', 'часто', 'появлялись', 'строки', 'вида', 'warn', 'token', 'update2025_version2', 'skipped', 'внутренние', 'отчеты', 'сохранялись', 'как', 'pdf', 'и', 'docx-файлы', 'интересно', 'что', 'в', 'текстах', 'часто', 'встречались', 'гибридные', 'конструкции', 'fake', 'news', 'инфодемия', 'post-truth', 'эпоха', 'fact-checking', 'soft', 'power', 'hard', 'skills', 'vs', 'soft', 'skills', 'некоторые', 'респонденты', 'писали', 'это', 'было', 'totally', 'unexpected', 'я', 'апдейтнул', 'файл', 'вчера', 'в', 'meeting', 'перенесли', 'на', 'monday', 'в', 'ит

**Задание 4. Удаление стоп-слов**

Выведите 2 списка - 1. Очищенных токенов, 2. Список удаленных стоп-слов

In [36]:
russian_stopwords = set(stopwords.words('russian'))

filtered_tokens = []
removed_stopwords = []

for token in tokenized_text:
    if token in russian_stopwords:
        removed_stopwords.append(token)
    else:
        filtered_tokens.append(token)

print(filtered_tokens)
print(removed_stopwords)

['году', 'команда', 'researchlab_42', 'запустила', 'пилотный', 'проект', 'digital', 'culture', 'mapping', 'цель', 'которого', 'анализ', 'user-generated', 'content', 'соцсетей', 'мессенджеров', 'словам', 'project', 'manager', 'kpi', 'выросли', 'сравнению', 'q4', 'несмотря', 'budget', 'cut', 'основные', 'данные', 'собирались', 'api', 'затем', 'обрабатывались', 'python', 'логах', 'часто', 'появлялись', 'строки', 'вида', 'warn', 'token', 'update2025_version2', 'skipped', 'внутренние', 'отчеты', 'сохранялись', 'pdf', 'docx-файлы', 'интересно', 'текстах', 'часто', 'встречались', 'гибридные', 'конструкции', 'fake', 'news', 'инфодемия', 'post-truth', 'эпоха', 'fact-checking', 'soft', 'power', 'hard', 'skills', 'vs', 'soft', 'skills', 'некоторые', 'респонденты', 'писали', 'это', 'totally', 'unexpected', 'апдейтнул', 'файл', 'вчера', 'meeting', 'перенесли', 'monday', 'итоговом', 'summary', 'отмечалось', 'частотность', 'аббревиатур', 'типа', 'ai', 'nlp', 'llm', 'gpt', 'eu', 'usa', 'превышает', 'н

**Задание 5. Лемматизация и стемминг**

Примените к токенам алгоритмы лемматизации и стемминга. Выведите 2 списка - 1. Лемматизированные токены 2. Стемматизированные токены

In [37]:
english_stemmer = SnowballStemmer("english")
russian_stemmer = SnowballStemmer("russian")
nltk.download('wordnet')

mystem = Mystem()
lemmatizer_en = WordNetLemmatizer()

def is_cyrillic(token):
    return any('а' <= char <= 'я' or char == 'ё' for char in token)

def is_latin(token):
    return any('a' <= char <= 'z' for char in token)

def stem_words(filtered_tokens):

    stemmed_tokens = []

    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                stem = russian_stemmer.stem(token)
                stemmed_tokens.append(stem)

            elif is_latin(token):
                stem = english_stemmer.stem(token)
                stemmed_tokens.append(stem)
    return stemmed_tokens

def lemm_words(filtered_tokens):

    lemmatized_tokens = []

    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                lemma = mystem.lemmatize(token)[0]
                lemmatized_tokens.append(lemma)

            elif is_latin(token):
                lemma = lemmatizer_en.lemmatize(token)
                lemmatized_tokens.append(lemma)
    return lemmatized_tokens

stemmed_tokens = stem_words(filtered_tokens)
lemmatized_tokens = lemm_words(filtered_tokens)

print('Стемматизированные токены: ', stemmed_tokens)
print('Лемматизированные токены: ', lemmatized_tokens)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Стемматизированные токены:  ['год', 'команд', 'запуст', 'пилотн', 'проект', 'digit', 'cultur', 'map', 'цел', 'котор', 'анализ', 'content', 'соцсет', 'мессенджер', 'слов', 'project', 'manag', 'kpi', 'выросл', 'сравнен', 'несмотр', 'budget', 'cut', 'основн', 'дан', 'собира', 'api', 'зат', 'обрабатыва', 'python', 'лог', 'част', 'появля', 'строк', 'вид', 'warn', 'token', 'skip', 'внутрен', 'отчет', 'сохраня', 'pdf', 'интересн', 'текст', 'част', 'встреча', 'гибридн', 'конструкц', 'fake', 'news', 'инфодем', 'эпох', 'soft', 'power', 'hard', 'skill', 'vs', 'soft', 'skill', 'некотор', 'респондент', 'писа', 'эт', 'total', 'unexpect', 'апдейтнул', 'файл', 'вчер', 'meet', 'перенесл', 'monday', 'итогов', 'summari', 'отмеча', 'частотн', 'аббревиатур', 'тип', 'ai', 'nlp', 'llm', 'gpt', 'eu', 'usa', 'превыша', 'норм', 'академическ', 'дискурс', 'подтвержда', 'гипотез', 'гибридизац', 'научн', 'медийн', 'регистр']
Лемматизированные токены:  ['год', 'команда', 'запускать', 'пилотный', 'проект', 'digital',

**Задание 6. Напишите функцию для препроцессинга текста**

Объедините все шаги в одну функцию. Выведите результат с лемматизированным списком

In [60]:
def is_cyrillic(token):
    return any('а' <= ch <= 'я' or ch == 'ё' for ch in token.lower())

def is_latin(token):
    return any('a' <= ch <= 'z' for ch in token.lower())

def big_function(text):

    text = text.lower()
    text = re.sub(r'[©®™℠]+', ' ', text)
    text = re.sub(r'\([^)]*\)', ' ', text)
    text = text.replace('ё', 'е')
    text = re.sub(r'(\w)-(\w)', r'\1HYPHEN\2', text)
    text = re.sub(r'[^\w\sHYPHEN]', ' ', text)
    text = re.sub(r'HYPHEN', '-', text)
    text = re.sub(r'\b\d+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    tokens = re.findall(r'\b[\w\-]+\b', text)

    filtered_tokens = [token for token in tokens if token not in russian_stopwords]

    stemmed_tokens = []
    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                stemmed_tokens.append(russian_stemmer.stem(token))
            elif is_latin(token):
                stemmed_tokens.append(english_stemmer.stem(token))

    lemmatized_tokens = []
    for token in filtered_tokens:
        if token.isalpha():
            if is_cyrillic(token):
                lemmatized_tokens.append(mystem.lemmatize(token)[0])
            elif is_latin(token):
                lemmatized_tokens.append(lemmatizer_en.lemmatize(token))

    return {
        'tokens': tokens,
        'filtered_tokens': filtered_tokens,
        'stemmed_tokens': stemmed_tokens,
        'lemmatized_tokens': lemmatized_tokens
    }

result = big_function(text)

print('Лемматизированные токены:', result['lemmatized_tokens'])

Лемматизированные токены: ['год', 'команда', 'запускать', 'пилотный', 'проект', 'digital', 'culture', 'mapping', 'цель', 'который', 'анализ', 'content', 'соцсеть', 'мессенджер', 'слово', 'project', 'manager', 'kpi', 'вырастать', 'сравнение', 'несмотря', 'budget', 'cut', 'основной', 'данные', 'собираться', 'api', 'затем', 'обрабатываться', 'python', 'лог', 'часто', 'появляться', 'строка', 'вид', 'warn', 'token', 'skipped', 'внутренний', 'отчет', 'сохраняться', 'pdf', 'интересно', 'текст', 'часто', 'встречаться', 'гибридный', 'конструкция', 'fake', 'news', 'инфодемия', 'эпоха', 'soft', 'power', 'hard', 'skill', 'v', 'soft', 'skill', 'некоторые', 'респондент', 'писать', 'это', 'totally', 'unexpected', 'апдейтнуть', 'файл', 'вчера', 'meeting', 'перенести', 'monday', 'итоговый', 'summary', 'отмечаться', 'частотность', 'аббревиатура', 'тип', 'ai', 'nlp', 'llm', 'gpt', 'eu', 'usa', 'превышать', 'норма', 'академический', 'дискурс', 'подтверждать', 'гипотеза', 'гибридизация', 'научный', 'медийн